In [190]:
import pandas as pd
import numpy as np
import itertools
import json
import copy

class LitBot:
    EPSILON = 0.001
    def __init__(self, game_id, player_id, player_count, fake_game_seed = None):
        self.game_id = game_id
        self.player_id = player_id
        self.player_count = player_count

        self.encoding = {
            "set_id" : {
                0 : "lower_hearts",
                1 : "lower_diamonds",
                2 : "lower_spades",
                3 : "lower_clubs",
                4 : "upper_hearts",
                5 : "upper_diamonds",
                6 : "upper_spades",
                7 : "upper_clubs",
            },
            "set_id_type" : {
                0 : "lower",
                1 : "lower",
                2 : "lower",
                3 : "lower",
                4 : "upper",
                5 : "upper",
                6 : "upper",
                7 : "upper",
            },
            "card_id" : {
                "lower" : {
                    0 : "ace",
                    1 : "two",
                    2 : "three",
                    3 : "four",
                    4 : "five",
                    5 : "six",
                },
                "upper" : {
                    0 : "eight",
                    1 : "nine",
                    2 : "ten",
                    3 : "jack",
                    4 : "queen",
                    5 : "king",
                }
            }
        }
        if fake_game_seed is not None:
            (
                self._fake_initial_game_state_array,
                self._fake_initial_card_location_array
            ) =  self.initialize_fake_game(fake_game_seed)
            self.initialize_matrix(self.player_id, self._fake_initial_game_state_array[0])
   
    def initialize_fake_game(self, fake_game_seed):
        
        np.random.seed(fake_game_seed)
        card_location_array = np.random.choice(range(self.player_count), (8,6))

        game_state_array = np.zeros((self.player_count, 8, 6))
        for set_id, card_id in itertools.product(range(8), range(6)):
            game_state_array[
                card_location_array[set_id, card_id],
                set_id,        
                card_id
            ] = 1
        
        return game_state_array, card_location_array

    def initialize_matrix(self, player_id, player_array):
        
        self.truth_matrix = np.full((self.player_count, 8, 6), LitBot.EPSILON)
        self.truth_matrix[player_id, :, :] = player_array
        self.truth_matrix[[i for i in np.arange(self.player_count) if i != player_id ], :, :] -= player_array*LitBot.EPSILON
        
        self.inference_matrix = copy.deepcopy(self.truth_matrix)

        self.active_cards_matrix = np.zeros((8, 6))
        self.recent_card_array = np.full(8, -1)

        self.prob_matrix = np.zeros((0, self.player_count, 8, 6))
        self.prob_matrix = self.update_prob_matrix(self.truth_matrix)

        self.total_shannon_info = -np.log2(np.full((8,6), 1/self.player_count))
        self.shannon_info_matrix = self.update_shannon_info_matrix(self.prob_matrix)
        
        
    def update_prob_matrix(self, truth_matrix):

        prob_matrix = np.zeros((1, self.player_count, 8, 6))
        prob_matrix[0, :, :, :] += truth_matrix

        for set_id, card_id in itertools.product(range(0,8), range(0,6)):
            prob_matrix[0, :, set_id, card_id] = np.where(
                truth_matrix[:, set_id, card_id] == LitBot.EPSILON,
                LitBot.EPSILON/truth_matrix[:, set_id, card_id].sum(),
                truth_matrix[:, set_id, card_id]
            )
                
        # self.prob_matrix = prob_matrix
        return prob_matrix

    def update_shannon_info_matrix(self, prob_matrix):

        uncertainity_matrix = prob_matrix.max(axis=(0, 1), keepdims=True)
        shannon_info_matrix = self.total_shannon_info + np.log2(uncertainity_matrix)

        # self.shannon_info_matrix = shannon_info_matrix
        return shannon_info_matrix

    def update_active_cards(self, set_id, card_id, result):
        if result == 1:
            self.active_cards_matrix[set_id, card_id] = 0
        else:
            self.active_cards_matrix[set_id, card_id] = 1

    def update_inference_matrix(self, ask_player_id, set_id, card_id):
        # if self.active_cards_matrix[set_id].sum() > 0:
        #     if self.active_cards_matrix[set_id, card_id] == 0:
        #         self.inference_matrix[
        #             ask_player_id,
        #             set_id,
        #             np.where(self.active_cards_matrix[set_id] == 1)[0].tolist()
        #         ] = 1/self.active_cards_matrix[set_id].sum()
        
        if self.recent_card_array[set_id] != card_id:
            self.inference_matrix[
                ask_player_id,
                set_id,
                self.recent_card_array[set_id]
            ] = 1

    def update_game(self, game_action_dict, stop = True):
        
        if game_action_dict["action"] == "ask_card":

            ask_player_id = game_action_dict["by"]
            ans_player_id = game_action_dict["to"]
            set_id = game_action_dict["set_id"]
            card_id = game_action_dict["card_id"]
            result = game_action_dict["result"]

            if result == 1:
                self.truth_matrix[ask_player_id, set_id, card_id] = 1
                self.truth_matrix[[i for i in np.arange(self.player_count) if i != ask_player_id ], set_id, card_id] = 0

                self.inference_matrix[ask_player_id, set_id, card_id] = 1
                self.inference_matrix[[i for i in np.arange(self.player_count) if i != ask_player_id ], set_id, card_id] = 0
            else:
                self.truth_matrix[[ask_player_id, ans_player_id], set_id, card_id] = 0
                self.inference_matrix[[ask_player_id, ans_player_id], set_id, card_id] = 0
                
                self.update_inference_matrix(ask_player_id, set_id, card_id)

            self.update_active_cards(set_id, card_id, result)
            self.recent_card_array[set_id] = card_id
            self.prob_matrix = self.update_prob_matrix(self.truth_matrix)
            self.shannon_info_matrix = self.update_shannon_info_matrix(self.prob_matrix)

# truth_matrix (player_id, set_id, card_id)
# prob_matrix  (0, player_id, set_id, card_id)

In [191]:
action1 = {
    "action" : "ask_card",
    "by" : 0,
    "to" : 4,
    "set_id" : 0,
    "card_id" : 0,
    "result" : 1
}
action2 = {
    "action" : "ask_card",
    "by" : 0,
    "to" : 5,
    "set_id" : 3,
    "card_id" : 0,
    "result" : 1
}
action3 = {
    "action" : "ask_card",
    "by" : 0,
    "to" : 5,
    "set_id" : 2,
    "card_id" : 3,
    "result" : 0
}
action4 = {
    "action" : "ask_card",
    "by" : 1,
    "to" : 2,
    "set_id" : 2,
    "card_id" : 2,
    "result" : 1
}
action5 = {
    "action" : "ask_card",
    "by" : 1,
    "to" : 2,
    "set_id" : 3,
    "card_id" : 5,
    "result" : 0
}
action6 = {
    "action" : "ask_card",
    "by" : 2,
    "to" : 4,
    "set_id" : 2,
    "card_id" : 3,
    "result" : 1
}
action7 = {
    "action" : "ask_card",
    "by" : 3,
    "to" : 1,
    "set_id" : 3,
    "card_id" : 4,
    "result" : 0
}
action8 = {
    "action" : "ask_card",
    "by" : 4,
    "to" : 5,
    "set_id" : 3,
    "card_id" : 3,
    "result" : 0
}

In [192]:
bot1 = LitBot(0, 0, 6, 0)

In [193]:
bot1.update_game(action1)
bot1.update_game(action2)
bot1.update_game(action3)
bot1.update_game(action4)
bot1.update_game(action5)
bot1.update_game(action6)
bot1.update_game(action7)

In [194]:
bot1.update_game(action8)

In [98]:
    # def _get_random_generator_state(self):
    #     with open('randomiser_state.json', mode="r") as file:
    #         random_generator_state = tuple(json.loads(file.read()))
    #     self.random_generator_state = random_generator_state